#Intent Classification using LSTM
This use-case provides a demo of how LSTM can be used for Intent classification in texts.

##Workflow:

1.   Understanding the problem
2.   Reading the data and understanding it
3.   Data Preprocessing
4.   Build LSTM model
5.   Train & Evaluate the model

##1. Understanding the problem
Intent Classification is the automated association of text to a specific intention. For example: Let's say you are writing an email to one of the Airlines and the text of the same is 'Can you please cancel my ticket with PNR 123456'. The intent of the customer here is 'Cancellation of Air Ticket'.

The idea of this use case to introduce the concept of Intent classification and how can LSTM be used to solve this.

###Import the necessary libraries
Please load the following packages before you proceed further.


In [1]:
import numpy as np
import pandas as pd
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('atis_intents_train.csv',
                       names= ["target", "text"])

test= pd.read_csv('atis_intents_test.csv',
                       names= ["target", "text"])

In [17]:
train.shape

(4834, 2)

In [18]:
test.shape

(800, 2)

####This is how the data looks like

In [19]:
train.head(10) #Get Top 10 rows from train dataset

,target,text
0,atis_flight,want fli boston 838 arriv denver 1110 morn
1,atis_flight,flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,arriv time san francisco 755 flight leav washi...
3,atis_airfare,cheapest airfar tacoma orlando
4,atis_airfare,round trip fare pittsburgh philadelphia 1000 d...
5,atis_flight,need flight tomorrow columbu minneapoli
6,atis_aircraft,kind aircraft use flight cleveland dalla
7,atis_flight,show flight pittsburgh lo angel thursday
8,atis_flight,flight boston washington
9,atis_ground_service,kind ground transport avail denver


 Check the number of intents.

In [20]:
train['target'].value_counts() #Get counts of different types of target variable in train data. We will not be using this anywhere but it is just for the overview of the data.

atis_flight            3666
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: target, dtype: int64

### 3. Preprocessing the Data
We will be doing the following preprocessing steps to get the desired format of the data.

1. Perform One Hot Encoding on the target variable of both train & test datasets.
2. Convert the text into lower case.
3. Tokenize the words.
4. Remove stop words.
5. Perform stemming & normalization.
6. Convert texts into sequences.
7. Pad the sequences.


In [21]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1)) #We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s.

Perform One Hot Encoding on the target variable. The output of this step would be an array with 0s and 1s.

In [22]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

In [24]:
train_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
test_target_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Convert text to lower case

In [26]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

In [27]:
train.head()

,target,text
0,atis_flight,want fli boston 838 arriv denver 1110 morn
1,atis_flight,flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,arriv time san francisco 755 flight leav washi...
3,atis_airfare,cheapest airfar tacoma orlando
4,atis_airfare,round trip fare pittsburgh philadelphia 1000 d...


In [28]:
test.head()

,target,text
0,atis_flight,would like find flight charlott la vega make s...
1,atis_airfare,april first need ticket tacoma san jose depart 7
2,atis_flight,april first need flight go phoenix san diego
3,atis_flight,would like flight travel one way phoenix san d...
4,atis_flight,would like flight orlando salt lake citi april...


Next step is to tokenize the text. We use word_tokenize function from nltk library for this purpose.

In [29]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [30]:
#Output of the above exercise looks like this
train["text"]

0       [want, fli, boston, 838, arriv, denver, 1110, ...
1       [flight, avail, pittsburgh, baltimor, thursday...
2       [arriv, time, san, francisco, 755, flight, lea...
3                     [cheapest, airfar, tacoma, orlando]
4       [round, trip, fare, pittsburgh, philadelphia, ...
                              ...                        
4829    [airfar, flight, denver, pittsburgh, delta, ai...
4830               [flight, denver, baltimor, via, dalla]
4831                                [airlin, fli, denver]
4832    [continent, fli, boston, san, francisco, stop,...
4833              [delta, flight, denver, san, francisco]
Name: text, Length: 4834, dtype: object

Eliminate stop words. 'english' dictionary from nltk.corpus library is used for this purpose. We also remove punctuation along with the removal of stop words.

In [31]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["text"]= train.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["text"]= test.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

Stemming & Normalizing


*   Stemming helps in reducing the word to the root form.
*   Normalizing is the process of transforming text into a standard form. Eg: Gud will be converted to good etc.




In [32]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [34]:
train.head()

,target,text
0,atis_flight,want fli boston 838 arriv denver 1110 morn
1,atis_flight,flight avail pittsburgh baltimor thursday morn
2,atis_flight_time,arriv time san francisco 755 flight leav washi...
3,atis_airfare,cheapest airfar tacoma orlando
4,atis_airfare,round trip fare pittsburgh philadelphia 1000 d...


Tokenize

In [33]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

In [35]:
tokenized_train_data

[[42, 19, 2, 296, 17, 5, 376, 21],
 [1, 37, 8, 10, 45, 21],
 [17, 102, 4, 6, 377, 1, 12, 18],
 [32, 212, 142, 91],
 [36, 30, 16, 8, 11, 180, 134],
 [33, 1, 98, 174, 138],
 [164, 99, 108, 1, 100, 9],
 [3, 1, 8, 103, 104, 45],
 [1, 2, 18],
 [164, 27, 29, 37, 5],
 [3, 1, 9, 4, 6],
 [3, 1, 4, 78, 67, 43, 87],
 [32, 1, 2, 225],
 [1, 10, 113, 22],
 [3, 24, 31, 16, 2, 5],
 [3, 27, 29, 5],
 [1, 5, 8, 12, 113, 22, 132, 22],
 [33, 55, 1, 52, 12, 10, 9, 9, 2, 2, 10],
 [20, 41, 1, 2, 8, 45, 79, 204],
 [23, 14, 19, 5, 8, 53, 13],
 [3, 1, 4, 78, 67],
 [20, 15, 24, 31, 1, 53, 5, 10],
 [164, 190, 108, 39, 13],
 [28, 14, 55, 109, 5, 8, 7],
 [28, 14, 136, 1, 7, 5],
 [13, 89, 5, 8, 7],
 [3, 1, 2, 8, 25, 79, 204, 12, 2, 181, 69, 22],
 [7, 27, 29],
 [205, 33, 147, 9, 2, 17, 66],
 [3, 32, 36, 30, 16, 10, 9],
 [3, 1, 44, 46, 57, 171, 13, 84, 378],
 [1, 44, 46, 62],
 [20, 58, 1, 65, 11, 4, 6, 41, 1, 240],
 [33, 1, 9, 4, 6],
 [16, 1, 8, 11],
 [3, 13, 24, 31, 1],
 [1, 56, 139, 185, 60, 63],
 [28, 14, 187, 1, 8,

Then, we pad the sequences

In [36]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [37]:
train_data

array([[  0,   0,   0, ...,   5, 376,  21],
       [  0,   0,   0, ...,  10,  45,  21],
       [  0,   0,   0, ...,   1,  12,  18],
       ...,
       [  0,   0,   0, ...,  13,  19,   5],
       [  0,   0,   0, ...,   6,  51,   5],
       [  0,   0,   0, ...,   5,   4,   6]], dtype=int32)

In [39]:
train_data.shape

(4834, 20)

Let's build a 3 dim array. The dimensions are samples, steps and unique words.

In [40]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [42]:
trans_matrix_train.shape

(4834, 20, 653)

In [46]:
trans_matrix_test.shape

(800, 20, 653)

### 4. Build LSTM Model

Build a class lstm_model_class that has three methods(methods are similar to functions) in a class, they are:



1.   Build a lstm model
2.   Train the created lstm model on the train data
3.   Predict the output on the train data

Note: Building a Class with 3 methods helps in tying all these three functions to the same object at the same instance.

Process of Building a LSTM Model:



1.   Build an embedded layer with dimensions as number of steps and input dimensions.
2.   Build an LSTM layer with number of steps equal to memory units.  
3.   Then, build a dense layer which is fully connected layer that represents a matrix vector multiplication.
4.   Apply the relu function after normalization and scaling of the activations. This is the standard activation function used.
5.   Finally, build the output layer.

Note:
1. activation function for multi-class classification problem - softmax
2. loss function is categorical cross entropy.
3. performace metric - Area under the curve(AUC)
4. optimizer would be the Adam optimizer.





In [15]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.

      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros",
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros",
             kernel_regularizer= l2(l= kernel_reg),
             bias_regularizer= l2(l= bias_reg))(dense_layer_1) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1)

      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model


Build the model on the necessary inputs.
We define the number of steps as the , output shape and input dimensio appropriately.

In [16]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data.
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8.
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps,
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed.
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias.



### 5. Train & Evaluate the model
In our last step, we will train and evaluate the model and check the performance metrics.

In [ ]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 60) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs.

Epoch 1/60
121/121 [==============================] - 2s 13ms/step - loss: 7.2804 - auc: 0.7257 - val_loss: 5.8829 - val_auc: 0.6351
Epoch 2/60
121/121 [==============================] - 1s 8ms/step - loss: 4.1178 - auc: 0.9361 - val_loss: 3.4945 - val_auc: 0.9867
Epoch 3/60
121/121 [==============================] - 1s 8ms/step - loss: 2.4641 - auc: 0.9746 - val_loss: 2.5842 - val_auc: 0.9226
Epoch 4/60
121/121 [==============================] - 1s 8ms/step - loss: 1.6020 - auc: 0.9844 - val_loss: 1.4742 - val_auc: 0.9943
Epoch 5/60
121/121 [==============================] - 1s 8ms/step - loss: 1.1521 - auc: 0.9908 - val_loss: 0.9134 - val_auc: 0.9963
Epoch 6/60
121/121 [==============================] - 1s 8ms/step - loss: 0.9257 - auc: 0.9938 - val_loss: 0.6942 - val_auc: 0.9979
Epoch 7/60
121/121 [==============================] - 1s 8ms/step - loss: 0.7929 - auc: 0.9952 - val_loss: 0.6916 - val_auc: 0.9956
Epoch 8/60
121/121 [==============================] - 1s 8ms/step - loss: 0

In [ ]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.target, pred_train)) #Print the classification report

                     precision    recall  f1-score   support

  atis_abbreviation       0.96      1.00      0.98       147
      atis_aircraft       0.97      0.96      0.97        81
       atis_airfare       0.99      0.99      0.99       423
       atis_airline       0.98      0.96      0.97       157
        atis_flight       1.00      1.00      1.00      3666
   atis_flight_time       0.98      0.94      0.96        54
atis_ground_service       1.00      1.00      1.00       255
      atis_quantity       1.00      1.00      1.00        51

           accuracy                           0.99      4834
          macro avg       0.98      0.98      0.98      4834
       weighted avg       0.99      0.99      0.99      4834



F1 and weighted avg are excellent. We can now move to implement this model on test data and see how it is performing.

In [ ]:
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.target, pred_test)) #Print the classification report

                     precision    recall  f1-score   support

  atis_abbreviation       0.80      1.00      0.89        33
      atis_aircraft       0.83      0.56      0.67         9
       atis_airfare       0.98      1.00      0.99        48
       atis_airline       0.97      0.76      0.85        38
        atis_flight       0.99      0.99      0.99       632
   atis_flight_time       0.50      1.00      0.67         1
atis_ground_service       1.00      0.94      0.97        36
      atis_quantity       0.43      1.00      0.60         3

           accuracy                           0.97       800
          macro avg       0.81      0.91      0.83       800
       weighted avg       0.98      0.97      0.97       800



F1 and weighted avg are excellent. We can settle with this model.